# Amazon product catalog: understanding ratings


## Problem scope

What do Amazon reviews say about the product, and can reviews be used reliably to predict the product category?

## Open questions / workflow

1. Predict the rating based on item desc.: regression w/ language data

2. How well reviewed something is

3. Figure out product, product contents, product tags, document per row + brand + company type, product category, description

4. Probability that the thing we labeled is actually in that class?


## Data imports

### Libraries

In [ ]:
#basic ops
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, re

#other helper functions
from pandasql import sqldf
from math import pi

#show grid on plots
sns.set_style('darkgrid')

#display tables without ellipsis
pd.set_option('display.max_rows', None)


#NLP
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer

#import gensim.downloader as api #allows us to get word2vec anf glove embeddings that we need
#from gensim.models.word2vec import Word2Vec
#from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#modeling
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, \
                                    cross_val_score

### Product dataset

In [ ]:
#import ecom dataset
ecom = pd.read_csv('../data/amazon_co-ecommerce_sample.csv')

In [ ]:
#check data size
ecom.shape

In [ ]:
#check data types
ecom.info()

In [ ]:
#peek at dataframe
ecom.head()

## Cleaning

Let's clean up our columns, including price, so that they can converge properly.

### Remove extraneous characters

From the pound signs in the prices, to prices listed as ranges, we have some issues in our data formats. Let's fix these.

For price ranges, I will merely consume the lower bound of the price.

In [ ]:
#Substantial datatype conversion help here from https://pbpython.com/currency-cleanup.html

def clean_price(x):
    
    if isinstance(x, str):
#If the value is a string, then remove currency symbol, delimiters and anything 
#else following the price; otherwise, the value is numeric and can be converted as is.
        
        return(x.replace('£', '').replace(',', '').split(' - ')[0])
    #strip price of the pound sign
    #we need to get rid of price ranges. I will just consume the LOWER BOUND in case of a price range.

    return(x)

In [ ]:
#apply my price function to convert the price column type
ecom['price'] = ecom['price'].apply(clean_price).astype('float')

In [ ]:
#strip out "out of 5 stars" from average_review_rating
ecom['average_review_rating'] = ecom['average_review_rating'].str.strip(' out of 5 stars')
ecom['average_review_rating'].head(2)

In [ ]:
#strip out the word "new" from number_available_in_stock; we'll handle this during feature eng

In [ ]:
#strip out commas from number_of_reviews column
ecom['number_of_reviews'] = ecom['number_of_reviews'].str.replace(",", "")

In [ ]:
ecom['number_of_reviews'].sort_values(ascending=False)[:30]

In [ ]:
#double check that our digits converted correctly
ecom.loc[ecom['number_of_reviews']=='1040']

In [ ]:
#remove line breaks and other junk from review and customer q's columns

ecom.replace(
    to_replace=[r"\\t|\\n|\\r|//", "\t|\n|\r"], value=["",""],
    regex=True, inplace=True)

ecom.replace(r'[^a-zA-Z0-9]', ' ', regex=True, inplace=True)

### Converting object columns to numeric where applicable

We had 4 columns originally that seemed like good candidates for conversion to numeric:

* price
* number_of_reviews
* number_of_answered_questions
* average_review_rating

In [ ]:
#force num type on number_of_reviews, number_of_answered_questions, price, average_review_rating
ecom['number_of_answered_questions'] = ecom[['number_of_answered_questions']].apply(pd.to_numeric)

In [ ]:
#ecom['average_review_rating'] = ecom['average_review_rating'].astype(float)

In [ ]:
#first remove nulls for this to work
ecom.dropna(subset=['number_of_reviews'], inplace=True)

#convert
ecom['number_of_reviews'] = ecom['number_of_reviews'].astype(int)

In [ ]:
#confirm conversions
ecom.dtypes

### Convert price

A nuance observed here is that we have a complicating issue in our data set in that some prices are displayed as ranges.

There were two places where we could look at price: the price column, which we have now reduced to the lower bound for simplicity, as well as the seller column.

One way to think of the price variability is that each seller can charge a different price. However, the wild variability can also be explained by product condition.

For simplicity, we are going to look at the average price, and create a new column we can reuse, with that information:

In [ ]:
#Gwen's code -- map avg. price to column

def median_sale_price(x):
    if pd.isna(x):
        return np.nan
    
    try:
        return np.median(
            [float(y.split('"')[0]) for y in list(x.split('£')[1:])])
    
    except:
        nos = []
        for y in list(x.split('£')[1:]):
            try:
                nos.append(float(y.split('"')[0]))
            except:
                pass
        return np.median(nos)
    
ecom['median_sale_price'] = ecom['sellers'].map(median_sale_price)

### Imputing nulls

For most of the fields, imputing with a 0 seems to make sense -- no reviews is no different than 0 reviews.

For price, however, we'll want to impute with the mean, now that we've successfully converted it to numeric.

As for average review rating, in the absence of one, we will simply drop those few observations since that's a valuable column and I'd want to be careful extending any kind of mean to it, since it is possibly our target.

In [ ]:
#`dropna.()` nulls without an average_review_rating since we need those for training, and there are few
ecom.dropna(
    subset=['average_review_rating'], inplace=True)

In [ ]:
#fill price with average of column
ecom['price'] = ecom['price'].fillna(
    ecom['price'].mean())

In [ ]:
#fill in NaN's with 0's for everything else
ecom = ecom.fillna(0)
ecom.head(3)

In [ ]:
ecom.isnull().sum()

## EDA

Interesting possible questions for exploration:

* Is there a correlation between number of reviews and number of questions?
* Is there a relationship between the number of sellers and the star rating?
* Is there a relationship between the star rating and the number of reviews?
* Is there a relationship between the product description and the star rating?
* Is there a relationship between a product being sold as new vs. used and its rating?
* Is there a relaionship between the product price and average rating?
* Do certain categories rank higher than others?

In [ ]:
#what can we analyze, again?
ecom.columns

What is the central tendency for the numeric columns in our dataset?

In [ ]:
#look at central tendency
ecom.describe().round(2).T

From the basic descriptive stats, we can conclude the following: 

* The average product price is 20 British pounds, while the most expensive item is north of 2K
* Most products have about 9 product reviews and 1-2 customer questions
* The average rating is a mere 2 stars

### Reviews

Taking a look at our data, all of the customer reviews appear to actually be compacted into each observation:

In [ ]:
#look at single review
ecom['customer_reviews'][0]

In [ ]:
#groupby product ID to reveal number of verbal reviews per product ID
#use uniq_id, customer_reviews

#first get count of distinct user IDs

#let's use some SQL, baby
#docstring: https://pypi.org/project/pandasql/

#from Gwen:
#For customer_reviews column:
#-strip /s, \r and \n s, digits, punctuation (in that order), maybe also months since they're easy


from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

def pysqldf(q):
    #add this to your script if you get tired of calling locals()
    return sqldf(q, globals())

df = ecom


#group reviews by product ID

q = """
    SELECT DISTINCT uniq_id, SUM(customer_reviews) AS num_reviews
    -- count gets number obs., sum is what we actually want
           FROM df
           GROUP BY uniq_id
           ORDER BY num_reviews DESC
              
"""

print(pysqldf(q).head(15))

#compare to what's reported as number of reviews per product
pd.DataFrame(
    ecom.groupby('uniq_id').sum(
    )['number_of_reviews'].sort_values(ascending=False))

Let's take a look visually at the distribution of customer reviews for a given product.

While not 100% predictive, this is one gauge or proxy we have about sales as a popularity indicator (with the actual rating being the other).

In [ ]:
plt.figure(figsize=(12,8))
plt.title('Distribution of product reviews for 150 random products', size=15)
plt.yticks([5, 10, 15, 20, 30, 40, 50, 100], size = 12)
plt.ylabel('Number of reviews', size=14)
plt.xlabel('Density = concentration of reviews per product', size=14)
ecom['number_of_reviews'][:150].plot(kind='box');
plt.savefig('../assets/concentration_reviews.jpg')

In [ ]:
#map no. reviews vs. no customer q's as ratio
ecom['review_to_q_ratio'] = ecom['number_of_reviews']/ecom['number_of_answered_questions']
pd.DataFrame(ecom['review_to_q_ratio'].value_counts())[:15]

### Categories

In [ ]:
#categories -- now we do not know, how the data were filtered
#during the scrape
cats = pd.DataFrame(
    ecom['amazon_category_and_sub_category'].value_counts())
print(cats.count())
cats[:50]

_It looks like this dataset favors toys. It is possible -- since we do not have the methodology used during scraping availed to us -- that the products have already been filtered for specific categories._

In [ ]:
#Do certain categories rank higher than others? - SPIDER CHART THIS!
q = """
    SELECT DISTINCT amazon_category_and_sub_category AS category,
        average_review_rating AS star_rating
           FROM df
           WHERE star_rating <> 0
           GROUP BY category
           ORDER BY star_rating DESC
              
"""

rating_by_cat = pd.DataFrame(pysqldf(q))

print('Top rated categories')
rating_by_cat.set_index('category')[:15]

In [ ]:
print('Lowest rated categories')
rating_by_cat.set_index('category').tail(15)

### Manufacturers

In [ ]:
#manufs
manufs = pd.DataFrame(ecom['manufacturer'].value_counts())
print(manufs.count())
manufs[:25]

_As with categories, we see a predominance / prepondrance of gaming items here._

### Price

In [ ]:
#look at price per manuf
def pysqldf(q):
    #add this to your script if you get tired of calling locals()
    return sqldf(q, globals())

df = ecom


q = """
    SELECT DISTINCT manufacturer, AVG(median_sale_price) AS avg_price
           FROM df
           GROUP BY manufacturer
           ORDER BY avg_price DESC
              
"""

price_by_manuf = pd.DataFrame(pysqldf(q))

price_by_manuf.set_index(
    'manufacturer')[:15].plot(kind = 'barh');
plt.title(
    'Top 15 priciest Amazon toy/game manufacturers (sample)',
size=15)
plt.xlabel('Price', size=14);
plt.ylabel('Manufacturer', size=14)
plt.yticks(size=12, rotation=45);
plt.savefig('../assets/top_manuf_by_price.jpg')

In [ ]:
#price dist -- although might be more meaningful to do per category or manuf
plt.figure(figsize=(12,8))
ecom['price'][:150].plot(kind='box');
plt.xticks([]);
plt.ylabel('Price', size = 14)
plt.yticks(size=12)
plt.xlabel('Density = concentration of products', size=14)
plt.title('Price distribution of a random sample of Amazon items (in BP)', size=15);
plt.savefig('../assets/price_concentrate.jpg')

### New vs used items

NOTE: this is the number of product listings that show at least one new and at least one used product.

In [ ]:
ecom['number_available_in_stock'].value_counts()[:10]
#replace w/ a search for ~like new, ~like used, and then get a count: use `string.find()`

#Gwen: use `str.contains()` rather than `str.find`, which looks for index

new = ecom['number_available_in_stock'].str.contains('new').sum()
used = ecom['number_available_in_stock'].str.contains('used').sum()

print(f'There are {new} new items sold and {used} used')

In [ ]:
#new vs used (once new / used are split)
#sns.displot(data=ecom, x='number_of_reviews', kde=True, hue=''); #fill in w/ new vs used

### Sellers

In [ ]:
#count of sellers
ecom.groupby(['sellers']).count() #REDO this as a dict comp if desired

In [ ]:
##Gwen's code - number of different sellers for a single product:
ecom['sellers'][0].split('Seller_price_')[-1][0]

In [ ]:
#Gwen's code
# all the prices for a single item
# map to create column
[float(x.split(
    '"')[0].replace(
    "'",'')) for x in ecom['sellers'][0].split('£')[1:]]

_Diff. conditions._

In [ ]:
#of ratings per seller?

### Customers also bought...

In [ ]:
#whether ratings affect also bought products? but need to extract url...

### Relationships

In [ ]:
sns.pairplot(data=ecom, hue ='average_review_rating');
plt.title('Relationships between products and ratings')
plt.savefig('../assets/pairplot.jpg')

In [ ]:
#corr
corr = ecom.corr()
corr.round(2)

In [ ]:
#corr plot
#ref from https://seaborn.pydata.org/examples/many_pairwise_correlations.html

sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, annot=True, linewidths=.5)

plt.title(
    'Correlation between the star rating and other product attributes',
size = 15);
plt.xticks(size=12, rotation=55);
plt.yticks(size=12);
plt.savefig('../assets/Corr.jpg');

## Feature engineering

We need to convert things like ratings so that the modeling works correctly.

The plan:

* The ratings column is our target
* We need to categorize manufacturers and / or categories

In [ ]:
#current shape
ecom.shape

In [ ]:
#one-hot encode manufs and product categories

ecom = pd.get_dummies(ecom, columns = [
    'manufacturer', #manufs
    'amazon_category_and_sub_category' #product categories
],drop_first=True)

# Caution--big printout!!

#ecom.head().T

In [ ]:
#NOW check shape...
ecom.shape

In [ ]:
#move out new vs. used items into sep. columns (binarize as new_only);
#or can merely filter by ...

## NLP

Open questions:

* Sentiment of reviews
* Does sentiment correspond to star rating?
* Most interesting question bi-grams
* Most interesting review bi-grams

In [ ]:
#vectorize text. NOTE: Run the RegEx to remove line breaks from customer review and
#questions columns, first!


In [ ]:
#NOW check shape
ecom.shape

### Sentiment analysis

## Modeling - linear regression

In [ ]:
#reviews
#price
#category

## Evaluation

## Conclusions & next steps